# Comparison Analysis: Salinas vs Indian Pines

This notebook compares the results of Multi-class PLS-DA classification on two hyperspectral remote sensing datasets:
- **Salinas**: Salinas Valley, California (54,129 samples, 204 bands, 16 classes)
- **Indian Pines**: Northwestern Indiana (10,249 samples, 200 bands, 16 classes)

## Key Questions:
1. How do the two datasets compare in terms of classification difficulty?
2. Which classes are easiest/hardest to classify in each dataset?
3. How does class imbalance affect performance?
4. What can we learn about wavelength importance across datasets?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Load results
salinas_overall = pd.read_csv('salinas_results/overall_metrics.csv')
salinas_class = pd.read_csv('salinas_results/class_metrics.csv')

indian_overall = pd.read_csv('indian_pines_results/overall_metrics.csv')
indian_class = pd.read_csv('indian_pines_results/class_metrics.csv')

print("✓ Data loaded successfully")

## 1. Overall Performance Comparison

In [ ]:
# Create comparison DataFrame
comparison = pd.DataFrame({
    'Metric': salinas_overall['Metric'],
    'Salinas': salinas_overall['Value'],
    'Indian_Pines': indian_overall['Value']
})

print("Overall Metrics Comparison")
print("=" * 70)
print(f"{'Metric':<25} {'Salinas':>15} {'Indian Pines':>15} {'Difference':>15}")
print("-" * 70)
for _, row in comparison.iterrows():
    diff = row['Salinas'] - row['Indian_Pines']
    print(f"{row['Metric']:<25} {row['Salinas']:>15.4f} {row['Indian_Pines']:>15.4f} {diff:>+15.4f}")
print("=" * 70)

In [ ]:
# Visualize overall performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Key metrics comparison
key_metrics = ['Overall_Accuracy', 'Cohens_Kappa', 'Macro_F1', 'Weighted_F1']
key_comparison = comparison[comparison['Metric'].isin(key_metrics)]

x = np.arange(len(key_metrics))
width = 0.35

axes[0].bar(x - width/2, key_comparison['Salinas'], width, label='Salinas', alpha=0.8)
axes[0].bar(x + width/2, key_comparison['Indian_Pines'], width, label='Indian Pines', alpha=0.8)
axes[0].set_xlabel('Metric')
axes[0].set_ylabel('Value')
axes[0].set_title('Key Performance Metrics Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels([m.replace('_', ' ') for m in key_metrics], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Precision/Recall/F1 comparison
prf_metrics = ['Macro_Precision', 'Macro_Recall', 'Macro_F1']
prf_comparison = comparison[comparison['Metric'].isin(prf_metrics)]

x2 = np.arange(len(prf_metrics))
axes[1].bar(x2 - width/2, prf_comparison['Salinas'], width, label='Salinas', alpha=0.8)
axes[1].bar(x2 + width/2, prf_comparison['Indian_Pines'], width, label='Indian Pines', alpha=0.8)
axes[1].set_xlabel('Metric')
axes[1].set_ylabel('Value')
axes[1].set_title('Macro-averaged Metrics Comparison')
axes[1].set_xticks(x2)
axes[1].set_xticklabels([m.replace('_', ' ').replace('Macro ', '') for m in prf_metrics])
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('comparison_overall_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Overall comparison plots generated")

## 2. Per-class Performance Analysis

In [ ]:
# Add dataset identifier
salinas_class['Dataset'] = 'Salinas'
indian_class['Dataset'] = 'Indian Pines'

# Combine for comparison
combined = pd.concat([salinas_class, indian_class], ignore_index=True)

print(f"Salinas - Average F1: {salinas_class['F1_Score'].mean():.4f}")
print(f"Salinas - Std F1: {salinas_class['F1_Score'].std():.4f}")
print(f"\nIndian Pines - Average F1: {indian_class['F1_Score'].mean():.4f}")
print(f"Indian Pines - Std F1: {indian_class['F1_Score'].std():.4f}")

In [ ]:
# Per-class F1-score comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Salinas
salinas_sorted = salinas_class.sort_values('F1_Score', ascending=True)
axes[0].barh(range(len(salinas_sorted)), salinas_sorted['F1_Score'], alpha=0.8)
axes[0].set_yticks(range(len(salinas_sorted)))
axes[0].set_yticklabels([f"C{row['Class']}: {row['Class_Name'][:25]}" for _, row in salinas_sorted.iterrows()], fontsize=9)
axes[0].set_xlabel('F1-Score')
axes[0].set_title('Salinas: Per-class F1-Score (sorted)')
axes[0].grid(axis='x', alpha=0.3)
axes[0].axvline(salinas_class['F1_Score'].mean(), color='red', linestyle='--', label=f'Mean: {salinas_class["F1_Score"].mean():.3f}')
axes[0].legend()

# Indian Pines
indian_sorted = indian_class.sort_values('F1_Score', ascending=True)
axes[1].barh(range(len(indian_sorted)), indian_sorted['F1_Score'], alpha=0.8, color='orange')
axes[1].set_yticks(range(len(indian_sorted)))
axes[1].set_yticklabels([f"C{row['Class']}: {row['Class_Name'][:25]}" for _, row in indian_sorted.iterrows()], fontsize=9)
axes[1].set_xlabel('F1-Score')
axes[1].set_title('Indian Pines: Per-class F1-Score (sorted)')
axes[1].grid(axis='x', alpha=0.3)
axes[1].axvline(indian_class['F1_Score'].mean(), color='red', linestyle='--', label=f'Mean: {indian_class["F1_Score"].mean():.3f}')
axes[1].legend()

plt.tight_layout()
plt.savefig('comparison_per_class_f1.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Per-class F1-score plots generated")

## 3. Class Imbalance Analysis

In [ ]:
# Analyze relationship between class size and F1-score
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Salinas
axes[0].scatter(salinas_class['Support'], salinas_class['F1_Score'], alpha=0.7, s=100)
for _, row in salinas_class.iterrows():
    axes[0].annotate(f"C{row['Class']}", (row['Support'], row['F1_Score']), 
                     fontsize=8, alpha=0.7, xytext=(5, 5), textcoords='offset points')
axes[0].set_xlabel('Class Support (# samples)')
axes[0].set_ylabel('F1-Score')
axes[0].set_title('Salinas: F1-Score vs Class Size')
axes[0].grid(alpha=0.3)

# Indian Pines
axes[1].scatter(indian_class['Support'], indian_class['F1_Score'], alpha=0.7, s=100, color='orange')
for _, row in indian_class.iterrows():
    axes[1].annotate(f"C{row['Class']}", (row['Support'], row['F1_Score']), 
                     fontsize=8, alpha=0.7, xytext=(5, 5), textcoords='offset points')
axes[1].set_xlabel('Class Support (# samples)')
axes[1].set_ylabel('F1-Score')
axes[1].set_title('Indian Pines: F1-Score vs Class Size')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('comparison_imbalance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate correlation
salinas_corr = salinas_class[['Support', 'F1_Score']].corr().iloc[0, 1]
indian_corr = indian_class[['Support', 'F1_Score']].corr().iloc[0, 1]

print(f"\nCorrelation between class size and F1-score:")
print(f"  Salinas: {salinas_corr:.4f}")
print(f"  Indian Pines: {indian_corr:.4f}")
print(f"\nInterpretation: {'Positive' if indian_corr > 0 else 'Negative'} correlation in Indian Pines")
print(f"indicates that larger classes tend to have {'higher' if indian_corr > 0 else 'lower'} F1-scores.")

## 4. Best and Worst Performing Classes

In [ ]:
print("="*80)
print("TOP 5 BEST PERFORMING CLASSES")
print("="*80)

print("\nSalinas:")
print("-" * 80)
salinas_top5 = salinas_class.nlargest(5, 'F1_Score')[['Class', 'Class_Name', 'F1_Score', 'Support']]
for _, row in salinas_top5.iterrows():
    print(f"  Class {row['Class']:2d} - {row['Class_Name']:35s} F1: {row['F1_Score']:.4f} (n={row['Support']:,})")

print("\nIndian Pines:")
print("-" * 80)
indian_top5 = indian_class.nlargest(5, 'F1_Score')[['Class', 'Class_Name', 'F1_Score', 'Support']]
for _, row in indian_top5.iterrows():
    print(f"  Class {row['Class']:2d} - {row['Class_Name']:35s} F1: {row['F1_Score']:.4f} (n={row['Support']:,})")

print("\n" + "="*80)
print("TOP 5 WORST PERFORMING CLASSES")
print("="*80)

print("\nSalinas:")
print("-" * 80)
salinas_bottom5 = salinas_class.nsmallest(5, 'F1_Score')[['Class', 'Class_Name', 'F1_Score', 'Support']]
for _, row in salinas_bottom5.iterrows():
    print(f"  Class {row['Class']:2d} - {row['Class_Name']:35s} F1: {row['F1_Score']:.4f} (n={row['Support']:,})")

print("\nIndian Pines:")
print("-" * 80)
indian_bottom5 = indian_class.nsmallest(5, 'F1_Score')[['Class', 'Class_Name', 'F1_Score', 'Support']]
for _, row in indian_bottom5.iterrows():
    print(f"  Class {row['Class']:2d} - {row['Class_Name']:35s} F1: {row['F1_Score']:.4f} (n={row['Support']:,})")

## 5. Confusion Matrix Visualization

In [ ]:
# Display confusion matrices side by side
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Load confusion matrix images
salinas_cm = Image.open('salinas_results/confusion_matrix_normalized.png')
indian_cm = Image.open('indian_pines_results/confusion_matrix_normalized.png')

axes[0].imshow(salinas_cm)
axes[0].axis('off')
axes[0].set_title('Salinas - Normalized Confusion Matrix', fontsize=14, pad=10)

axes[1].imshow(indian_cm)
axes[1].axis('off')
axes[1].set_title('Indian Pines - Normalized Confusion Matrix', fontsize=14, pad=10)

plt.tight_layout()
plt.savefig('comparison_confusion_matrices.png', dpi=200, bbox_inches='tight')
plt.show()

print("✓ Confusion matrices displayed")

## 6. Summary and Insights

In [ ]:
print("="*80)
print("SUMMARY OF FINDINGS")
print("="*80)

acc_diff = comparison[comparison['Metric'] == 'Overall_Accuracy']['Salinas'].values[0] - \
           comparison[comparison['Metric'] == 'Overall_Accuracy']['Indian_Pines'].values[0]

print(f"\n1. Overall Performance:")
print(f"   - Salinas outperforms Indian Pines by {acc_diff*100:.2f}% in overall accuracy")
print(f"   - Both datasets show moderate agreement (Kappa > 0.4)")

print(f"\n2. Class Imbalance Impact:")
salinas_imbalance = salinas_class['Support'].max() / salinas_class['Support'].min()
indian_imbalance = indian_class['Support'].max() / indian_class['Support'].min()
print(f"   - Salinas imbalance ratio: {salinas_imbalance:.1f}:1")
print(f"   - Indian Pines imbalance ratio: {indian_imbalance:.1f}:1")
print(f"   - Indian Pines' higher imbalance correlates with lower performance")

print(f"\n3. Dataset Characteristics:")
print(f"   - Salinas: More balanced classes, agricultural vegetation focus")
print(f"   - Indian Pines: Severe imbalance (122:1), diverse crop types")

print(f"\n4. Recommendations for Improvement:")
print(f"   - For Indian Pines: Consider class balancing techniques (oversampling, SMOTE)")
print(f"   - For both: Implement CARS wavelength selection to reduce dimensionality")
print(f"   - Explore ensemble methods or deep learning for better small-class performance")

print("\n" + "="*80)

## Conclusion

This analysis demonstrates that:
1. **Dataset difficulty varies significantly**: Salinas is easier to classify than Indian Pines
2. **Class imbalance matters**: Indian Pines' extreme imbalance (122:1) severely impacts small class performance
3. **PLS-DA baseline established**: Both datasets now have baseline performance metrics for CARS wavelength selection
4. **Next steps**: Implement full CARS algorithm to select optimal wavelengths and potentially improve performance while reducing dimensionality